In [1]:
from bs4 import BeautifulSoup as bs

In [2]:
import requests

In [3]:
import pprint

In [4]:
import pandas as pd

In [5]:
url = 'https://roscontrol.com/category/produkti/'

In [6]:
headers = {'User-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36',
           'Accept': '*/*'}

In [7]:
cat_selection = requests.get(url, headers=headers)

In [8]:
soup = bs(cat_selection.text, 'html.parser')

In [9]:
cat_names = soup.find_all('div',attrs={'class':'catalog__category-name'})

In [10]:
for i in range(len(cat_names)):
    cat_names[i] = cat_names[i].string

In [11]:
cat_links = soup.find_all('a',attrs={'class':'catalog__category-item'})

In [12]:
for i in range(len(cat_links)):
    cat_links[i] = cat_links[i]['href'].replace('/category/produkti/', '')

In [13]:
categories = {
    "Наименование": cat_names,
    "Ссылка": cat_links
}

categories = pd.DataFrame(categories)

In [14]:
category = categories.loc[int(input(categories["Наименование"])), "Ссылка"]

0                       Молочные продукты 
1                         Мясные продукты 
2                                   Птица 
3                     Рыба и морепродукты 
4                     Вода, соки, напитки 
5                   Хлебобулочные изделия 
6                         Детское питание 
7                                    Яйца 
8                                 Бакалея 
9                         Горячие напитки 
10                   Кондитерские изделия 
11                    Алкогольные напитки 
12                                  Соусы 
13                          Полуфабрикаты 
14                     Растительное масло 
15                               Консервы 
16                                  Снеки 
17                   Фрукты, овощи, грибы 
18                              Маргарины 
19     Готовые блюда и кулинарные изделия 
Name: Наименование, dtype: object1


In [15]:
subcat_selection = requests.get(url + category, headers=headers)

In [16]:
soup = bs(subcat_selection.text, 'html.parser')

In [17]:
subcat_names = soup.find_all('div',attrs={'class':'catalog__category-name'})

if subcat_names:
    for i in range(len(subcat_names)):
        subcat_names[i] = subcat_names[i].string

    subcat_links = soup.find_all('a',attrs={'class':'catalog__category-item'})

    for i in range(len(subcat_links)):
        subcat_links[i] = subcat_links[i]['href'].replace('/category/produkti/' + category, '')

    subcategories = {
        "Наименование": subcat_names,
        "Ссылка": subcat_links
    }

    subcategories = pd.DataFrame(subcategories)
    subcategory = subcategories.loc[int(input(subcategories["Наименование"])), "Ссылка"]
else:
    subcategory = ''

0                             Колбаса вареная 
1      Полукопченая и варено-копченая колбаса 
2                         Сосиски и сардельки 
3                                     Ветчина 
4                                        Фарш 
5                             Мясные консервы 
6                          Мясная гастрономия 
7                                     Паштеты 
8                          Пельмени и хинкали 
9                    Аналоги мясных продуктов 
10                                       Мясо 
11                       Сырокопченая колбаса 
12                       Полуфабрикаты мясные 
13                         Субпродукты мясные 
Name: Наименование, dtype: object0


In [18]:
response = requests.get(url + category + subcategory, headers=headers)
soup = bs(response.text, 'html.parser')
pages = soup.find_all('a', attrs={'class':'page-num'})
if pages:
    pages_count = len(pages) - 1
else:
    pages_count = 1
print(pages_count)

4


In [19]:
for p in range(pages_count):
    params = {'page':p+1}
    response = requests.get(url + category + subcategory, headers=headers, params=params)
    soup = bs(response.text, 'html.parser')
    products = soup.find_all('div', attrs={'class':'wrap-product-catalog__item'})
    products_list = []
    for product in products:
        product_data = {}
        name = product.find('div', attrs={'class':'product__item-link'}).getText()
        link = 'https://roscontrol.com' + product.contents[1]['href']
        try:
            rating_status = product.find('div', attrs={'class':'rating-total-text'}).string
        except AttributeError:
            rating_status = False
        if rating_status == 'Общая оценка':
            total_rating = int(product.find('div', attrs={'class':'product-rating'}).contents[1].getText())
            rating_block = product.find('div', attrs={'class':'rating-block'})
            for row in rating_block.findChildren(recursive=False):
                parameter = row.findChildren(recursive=False)[0].findChildren(recursive=False)[0].getText()
                value = int(row.findChildren(recursive=False)[1].getText())
                product_data[parameter] = value
        elif rating_status == 'В чёрном списке':
            total_rating = 'В чёрном списке'
        elif not rating_status:
            total_rating = 'Нет информации о товаре'
        product_data['Наименование'] = name
        product_data['Ссылка'] = link
        product_data['Общий рейтинг'] = total_rating

        products_list.append(product_data)
    if p == 0:
        df_products = pd.DataFrame(products_list)
    else:
        df_products = pd.concat([df_products, pd.DataFrame(products_list)], axis=0, ignore_index=True)

In [20]:
df_products

,Безопасность,Натуральность,Пищевая ценность,Качество,Наименование,Ссылка,Общий рейтинг
0,70.0,30.0,10.0,56.0,"Колбаса ""Клинский"" ""Телячья""",https://roscontrol.com/product/kolbasa-klinski...,42
1,77.0,50.0,1.0,77.0,"""Окраина"", ""Русская""",https://roscontrol.com/product/kolbasa-russkay...,51
2,49.0,50.0,24.0,82.0,"Колбаса ""Окраина"", ""Докторская""",https://roscontrol.com/product/kolbasa-okraina...,51
3,56.0,40.0,13.0,76.0,"Колбаса ""Мираторг"", ""Докторская",https://roscontrol.com/product/kolbasa-mirator...,46
4,49.0,55.0,22.0,78.0,"""Велком"", ""Докторская""",https://roscontrol.com/product/velkom-doktorsk...,51
...,...,...,...,...,...,...,...
60,NaN,NaN,NaN,NaN,"""КампоМос"", ""Дикий кабан""",https://roscontrol.com/product/kolbasa-varenay...,В чёрном списке
61,NaN,NaN,NaN,NaN,"""ТД Рублевский"", ""Докторская""",https://roscontrol.com/product/td-rublevskiy-d...,В чёрном списке
62,NaN,NaN,NaN,NaN,"""Черкизовский МПЗ"", ""Докторская""",https://roscontrol.com/product/cherkizovskiy-m...,В чёрном списке
63,NaN,NaN,NaN,NaN,"Колбаса вареная ""Докторская"" ГОСТ",https://roscontrol.com/product/kolbasa-variena...,Нет информации о товаре


In [21]:
df_products.fillna(0, inplace = True)

In [22]:
df_products

,Безопасность,Натуральность,Пищевая ценность,Качество,Наименование,Ссылка,Общий рейтинг
0,70.0,30.0,10.0,56.0,"Колбаса ""Клинский"" ""Телячья""",https://roscontrol.com/product/kolbasa-klinski...,42
1,77.0,50.0,1.0,77.0,"""Окраина"", ""Русская""",https://roscontrol.com/product/kolbasa-russkay...,51
2,49.0,50.0,24.0,82.0,"Колбаса ""Окраина"", ""Докторская""",https://roscontrol.com/product/kolbasa-okraina...,51
3,56.0,40.0,13.0,76.0,"Колбаса ""Мираторг"", ""Докторская",https://roscontrol.com/product/kolbasa-mirator...,46
4,49.0,55.0,22.0,78.0,"""Велком"", ""Докторская""",https://roscontrol.com/product/velkom-doktorsk...,51
...,...,...,...,...,...,...,...
60,0.0,0.0,0.0,0.0,"""КампоМос"", ""Дикий кабан""",https://roscontrol.com/product/kolbasa-varenay...,В чёрном списке
61,0.0,0.0,0.0,0.0,"""ТД Рублевский"", ""Докторская""",https://roscontrol.com/product/td-rublevskiy-d...,В чёрном списке
62,0.0,0.0,0.0,0.0,"""Черкизовский МПЗ"", ""Докторская""",https://roscontrol.com/product/cherkizovskiy-m...,В чёрном списке
63,0.0,0.0,0.0,0.0,"Колбаса вареная ""Докторская"" ГОСТ",https://roscontrol.com/product/kolbasa-variena...,Нет информации о товаре


In [23]:
df_products.to_csv('df_products.csv', index=False)

In [24]:
dict_products =  df_products.to_dict('records')

# Урок 3

In [25]:
from pymongo import MongoClient
from pprint import pprint

In [26]:
client = MongoClient('127.0.0.1', 27017)

In [27]:
db = client['products']

In [28]:
saussages = db.saussages

In [29]:
def db_add(data):
    saussages.insert_many(data)

In [30]:
def db_add_or_update(data):
    saussages_links = []
    prod_added = 0
    for x in saussages.find({}):
        saussages_links.append(x['Ссылка'])
    for y in data:
        if y['Ссылка'] not in saussages_links:
            saussages.insert_one(y)

In [32]:
saussages.delete_many({})

In [33]:
db_add(dict_products)

In [34]:
db_add_or_update(dict_products)

In [35]:
min_value = int(input('Введите min значение: '))

Введите min значение: 70


In [36]:
def search_by(min_value):
    result = []
    for item in saussages.find({'$or':[{'Общий рейтинг': {'$gte': min_value}},{'Качество':{'$gte': min_value}}]}):
        result.append(item)
    return result

In [37]:
selection = search_by(50)

In [38]:
pprint(selection)

[{'_id': ObjectId('61292f3ad1dabf3e72ff450e'),
  'Безопасность': 70.0,
  'Качество': 56.0,
  'Наименование': 'Колбаса "Клинский" "Телячья"',
  'Натуральность': 30.0,
  'Общий рейтинг': 42,
  'Пищевая ценность': 10.0,
  'Ссылка': 'https://roscontrol.com/product/kolbasa-klinskiy-telyachya/'},
 {'_id': ObjectId('61292f3ad1dabf3e72ff450f'),
  'Безопасность': 77.0,
  'Качество': 77.0,
  'Наименование': '"Окраина", "Русская"',
  'Натуральность': 50.0,
  'Общий рейтинг': 51,
  'Пищевая ценность': 1.0,
  'Ссылка': 'https://roscontrol.com/product/kolbasa-russkaya-okraina/'},
 {'_id': ObjectId('61292f3ad1dabf3e72ff4510'),
  'Безопасность': 49.0,
  'Качество': 82.0,
  'Наименование': 'Колбаса "Окраина", "Докторская"',
  'Натуральность': 50.0,
  'Общий рейтинг': 51,
  'Пищевая ценность': 24.0,
  'Ссылка': 'https://roscontrol.com/product/kolbasa-okraina-doktorskaya/'},
 {'_id': ObjectId('61292f3ad1dabf3e72ff4511'),
  'Безопасность': 56.0,
  'Качество': 76.0,
  'Наименование': 'Колбаса "Мираторг", "